In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tslearn.clustering import KShape
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import sigma_gak, cdist_gak
from sklearn.metrics.cluster import adjusted_rand_score
import random
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import TimeSeriesKMeans
import xgboost
import gc
import Evrecsys
import sys
from itertools import combinations, groupby
from collections import Counter
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
import Apriori
import Evrecsys
import lightfm_form
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k

order_products_prior = pd.read_csv("order_products__prior.csv")

orders = pd.read_csv("orders.csv")

In [33]:
def get_scores(method,true,pred):
    sum_recall=0
    sum_precision=0
    sum_fscore=0
    n=len(true)
    for i in true.index:
        p,r,f1=Evrecsys.calculate_prf(true[i],pred[i])
        sum_precision=sum_precision+p
        sum_recall=sum_recall+r
        sum_fscore=sum_fscore+f1

    return [method,sum_recall/n,sum_precision/n ,sum_fscore/n]

def last_order(train, test):
    last_orders = train.groupby("user_id")["order_number"].aggregate(np.max)
    t = pd.merge(left=last_orders.reset_index(), right=train, how='inner', on=['user_id', 'order_number'])
    t_last_order = t.groupby('order_id').aggregate({'product_id': lambda x: list(x)})
    t_last_order = pd.merge(t_last_order, train[['order_id', 'user_id']], on='order_id')
    t_last_order = t_last_order.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

    test_history = test[test['reordered']==1].groupby('order_id').aggregate({'product_id': lambda x: list(x)})
    test_history = pd.merge(test_history, test[['order_id', 'user_id']], on='order_id')
    test_history = test_history.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

    t_last_order = pd.merge(t_last_order, test_history, on='user_id')
    t_last_order = t_last_order.sort_values('user_id')
    y_pred=t_last_order['product_id_x']
    y_true=t_last_order['product_id_y']

    return y_pred , y_true

def last_order_reorder(train, test):
    last_orders = train.groupby("user_id")["order_number"].aggregate(np.max)
    t = pd.merge(left=last_orders.reset_index(), right=train[train.reordered == 1], how='inner',
                 on=['user_id', 'order_number'])
    t_last_order = t.groupby('order_id').aggregate({'product_id': lambda x: list(x)})
    t_last_order = pd.merge(t_last_order, train[['order_id', 'user_id']], on='order_id')
    t_last_order = t_last_order.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

    test_history = test[test['reordered']==1].groupby('order_id').aggregate({'product_id': lambda x: list(x)})
    test_history = pd.merge(test_history, test[['order_id', 'user_id']], on='order_id')
    test_history = test_history.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

    t_last_order = pd.merge(t_last_order, test_history, on='user_id')
    t_last_order = t_last_order.sort_values('user_id')
    y_pred=t_last_order['product_id_x']
    y_true=t_last_order['product_id_y']

    return y_pred , y_true

def top10(train, test):
    top_reorder_train=train.groupby("product_id")["reordered"].aggregate({'Total_reorders': 'sum'})['Total_reorders'].sort_values(ascending=False).head(10)
    top_reorder_train=np.array(top_reorder_train.values)
    test_history=test[test['reordered']==1].groupby('order_id').aggregate({'product_id':lambda x: list(x)})
    return top_reorder_train, test_history['product_id']

#lightfm
from lightfm import LightFM
def use_lightfm(train, test):
    set1 = set(np.unique(train.product_id))
    set2 = set(np.unique(test.product_id))
    missing =  pd.DataFrame.from_dict(list(sorted(set1 - set2)))
    added =  pd.DataFrame.from_dict(list(sorted(set2 - set1)))
    
    for i in range(len(missing)): 
        a=missing[0][i]
        test=test.append({'product_id': a},ignore_index=True)
    for i in range(len(added)): 
        a=added[0][i]
        train=train.append({'product_id': a},ignore_index=True)
    

    train=train.fillna(0)
    test=test.fillna(0)
    
    grouped_train_i = train.groupby(["user_id", "product_id"])["reordered"].aggregate("sum").reset_index()
    grouped_test_i = test.groupby(["user_id", "product_id"])["reordered"].aggregate("sum").reset_index()
    
    interactions_i = lightfm_form.create_interaction_matrix(df = grouped_train_i,
                                         user_col = 'user_id',
                                         item_col = 'product_id',
                                         rating_col = 'reordered')
    
    interactions_test_i = lightfm_form.create_interaction_matrix(df = grouped_test_i,
                                         user_col = 'user_id',
                                         item_col = 'product_id',
                                         rating_col = 'reordered')
    
    mf_model = lightfm_form.runMF(interactions = interactions_i,
                 n_components = 30,loss = 'warp',epoch = 40,n_jobs = 4)
    
    test_history = test[test['reordered']==1].groupby('order_id').aggregate({'product_id': lambda x: list(x)})
    test_history = pd.merge(test_history, test[['order_id', 'user_id']], on='order_id')
    test_history = test_history.drop_duplicates(subset=['order_id', 'user_id'], keep='first')
    n_users, n_items = interactions_i.shape
    
    results=[]
    test_history['pred']=0
    for user_id in test_history['user_id']:
        print(user_id)
        recom = mf_model.predict(user_id, np.arange(n_items), num_threads=4)
        recom = pd.Series(recom)
        recom.sort_values(ascending=False, inplace=True)
        if(len(results) == 0):
            results = np.array(recom.iloc[0:10].index.values)
        else:
            results= np.vstack((results, recom.iloc[0:10].index.values))
    
    results_df = pd.DataFrame(data=results)
    columns= results_df.columns.values
    test_history['pred'] = results_df[columns].values.tolist()
           
    y_pred=test_history['pred']
    y_true=test_history['product_id']
    
    test_precision = precision_at_k(mf_model, sparse.csr_matrix(interactions_test_i.values), k=10).mean()
    test_recall = recall_at_k(mf_model, sparse.csr_matrix(interactions_test_i.values), k=10).mean()
    f_test=2 * test_precision * test_recall / (test_precision + test_recall)
    print(f_test)
    return y_pred , y_true, test_precision, test_recall, f_test
 
    
def apriori(train, test):
    train_orders_i = train.set_index('order_id')['product_id'].rename('item_id')
    test_orders_i = test.set_index('order_id')['product_id'].rename('item_id')
    
    item_name   = pd.read_csv('products.csv')
    item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
    rules_i = Apriori.association_rules(train_orders_i, 0.01)  
    rules_final_i = Apriori.merge_item_name(rules_i, item_name).sort_values('lift', ascending=False)
    display(rules_final_i)
    
    #Train set pairs
    train_pairs_gen_i=Apriori.get_item_pairs(train_orders_i)
    train_pairs_i = Apriori.freq(train_pairs_gen_i).to_frame("freqAB")
    train_pairs_i = train_pairs_i.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    train_pairs_i['pair']=train_pairs_i.item_A.astype(str).str.cat(train_pairs_i.item_B.astype(str), sep='-')
    
    #Test set pairs
    test_pairs_gen_i=Apriori.get_item_pairs(test_orders_i)
    test_pairs_i = Apriori.freq(test_pairs_gen_i).to_frame("freqAB")
    test_pairs_i = test_pairs_i.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    test_pairs_i['pair']=test_pairs_i.item_A.astype(str).str.cat(test_pairs_i.item_B.astype(str), sep='-')
    
    #Rules set pairs
    rules_i['pair']=rules_i.item_A.astype(str).str.cat(rules_i.item_B.astype(str), sep='-')
    
    test_pair_set_i=set(np.unique(test_pairs_i.pair))
    train_pair_set_i=set(np.unique(train_pairs_i.pair))
    rules_pair_set_i=set(np.unique(rules_i.pair))

    #TP= Pairs that exist in a priori pred and test
    tp=len(list(test_pair_set_i&rules_pair_set_i))

    #TN= pairs that exists train set but not in test
    tn=len(list(test_pair_set_i-train_pair_set_i))

    #FN= Pairs that exists in test but not in a priori
    fn=len(list(rules_pair_set_i-test_pair_set_i))

    #FP= Pairs that exists in a priori but not in test
    fp=len(list(test_pair_set_i-rules_pair_set_i))
    
    return tp, tn, fn, fp 

In [34]:
orders=orders.loc[orders['eval_set']=='prior']
users_list=np.unique(orders.user_id)

In [35]:
random.Random(4).shuffle(users_list)
users_A=users_list[1:50000]
users_B=users_list[50001:100000]
users_C=users_list[100001:150000]
users_D=users_list[150001:]

In [36]:
users_group=[users_A, users_B, users_C, users_D]

In [37]:
results=[]
for group in users_group:
    print(group)
    orders_set=orders[orders["user_id"].isin(group)]
    print(np.shape(orders))
    idx_test = orders_set.groupby(['user_id'])['order_number'].transform(max) == orders_set['order_number']
    orders_set_test=orders_set[idx_test]
    print(np.shape(orders_set_test))

    orders_set_testlist=np.unique(orders_set_test.order_id)
    orders_set_train = orders_set[-orders_set["order_id"].isin(orders_set_testlist)]
    orders_set_trainlist=np.unique(orders_set_train.order_id)
    train= order_products_prior[order_products_prior["order_id"].isin(orders_set_trainlist)]
    test=order_products_prior[order_products_prior["order_id"].isin(orders_set_testlist)]
    print(np.shape(train))
    train = pd.merge(train, orders_set_train, on='order_id', how='left')
    test=pd.merge(test, orders_set_test, on='order_id', how='left')
    print(np.shape(test))
    y_pred_last_order , y_true_last_order= last_order(train, test)
    print('finish last order')
    y_pred_last_reorder , y_true_last_reorder= last_order_reorder(train, test)
    print('finish last reorder')
    y_pred_top10 , y_true_top10=top10(train, test)
    print('finish top 10')
    y_pred_lightfm , y_true_lightfm, test_precision_light, test_recall_light, f_test_light=use_lightfm(train, test)
    print('finish lightfm')
    
    results.append([group,group ,group, group])
    
    #Top 10
    sum_recall=0
    sum_precision=0
    sum_fscore=0
    n=len(y_true_top10)
    print(n)
    for i in y_true_top10.index:
        p,r,f1=Evrecsys.calculate_prf(y_true_top10[i],y_pred_top10)
        sum_precision=sum_precision+p
        sum_recall=sum_recall+r
        sum_fscore=sum_fscore+f1

    results.append(['top10',sum_recall/n,sum_precision/n ,sum_fscore/n])
    
    #Last order
    scores_lo=get_scores('last order',y_true_last_order,y_pred_last_order)
    results.append(scores_lo)
    
    #last reorder
    scores_re=get_scores('last reorder',y_true_last_reorder,y_pred_last_reorder)
    results.append(scores_re)
    
    #lightfm
    scores_li=get_scores('lightfm',y_true_lightfm,y_pred_lightfm)
    results.append(scores_li)
    
    #apriori
    tp, tn, fn, fp=apriori(train, test)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1=2*(recall*precision)/(recall+precision)
    results.append(['apriori',recall, precision, f1])
    
    
    

[127496 122627 183297 ... 155152 102625  18080]
(3214874, 7)
(49999, 7)
(7330409, 4)
(517822, 10)
finish last order
finish last reorder


/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


finish top 10


MemoryError: 

In [32]:
result = pd.DataFrame(results,columns=['method', 'recall', 'precision', 'fscore'])
result

,method,recall,precision,fscore
0,group,group,group,group
1,top10,1.7073e-05,1.29104e-05,1.38951e-05
2,last order,0.47889,0.309155,0.340483
3,last reorder,0.375879,0.40496,0.354944
4,apriori,0.993865,0.0173695,0.0341424
5,group,group,group,group
6,top10,7.79547e-05,5.61082e-05,5.74155e-05
7,last order,0.48113,0.310878,0.34222
8,last reorder,0.375931,0.40444,0.354369
9,apriori,0.994209,0.0177837,0.0349423
